# Step 2: visualization

In [ ]:
import sys
import os
from os.path import join
import time
from datetime import datetime
import importlib
import numpy as np
import pandas as pd
import h5py
import sympy
from tqdm.notebook import tqdm
from tqdm.notebook import trange
import matplotlib as mpl
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib import patches
import plotly.graph_objs as go
from ipywidgets import interactive
from ipywidgets import widgets
from IPython.display import display
from IPython.display import clear_output
import proplot as pplt

sys.path.append('../..')
from tools import analysis as ba
from tools import energyVS06 as energy
from tools import image_processing as ip
from tools import plotting as mplt
from tools import utils
from tools.utils import project

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['pdf.fonttype'] = 42  # TrueType font... for NAPAC-2022
pplt.rc['savefig.dpi'] = 300  # default is 1000...

## Load data 

In [ ]:
folder = '_saved/2022-03-17-VS06/'

In [ ]:
info = utils.load_pickle(join(folder, 'info.pkl'))
info

In [ ]:
filename = info['filename']
coords = utils.load_stacked_arrays(join(folder, f'coords_{filename}.npz'))
shape = tuple([len(c) for c in coords])
print('shape:', shape)

In [ ]:
f = np.memmap(join(folder, f'f_{filename}.mmp'), shape=shape, dtype='float', mode='r')

In [ ]:
f_max = np.max(f)
f_min = np.min(f)
if f_min < 0.0:
    print(f'min(f) = {f_min}')
    print('Clipping to zero.')
    f = np.clip(f, 0.0, None)

In [ ]:
dims = ["x", "x'", "y", "y'", "w"]
units = ["mm", "mrad", "mm", "mrad", "MeV"]
dims_units = [f'{d} [{u}]' for d, u in zip(dims, units)]
labels = dims_units
prof_kws = dict(lw=0.5, alpha=0.7, color='white', scale=0.12, kind='step')

Center the grid on the beam centroid. This will make it easier to compare with simulation.

In [ ]:
proj = []
for i in range(f.ndim):
    p = utils.project(f, i)
    p = p / np.sum(p)
    proj.append(p)
    
centroid = [np.average(c, weights=p) for c, p in zip(coords, proj)]
for i in range(5):
    coords[i] = coords[i] - centroid[i]

Crop the array if you want to.

In [ ]:
crop = (
    (5, f.shape[0] - 0),
    (1, f.shape[1] - 3),
    (0, f.shape[2] - 0),
    (15, f.shape[3] - 15),
    (20, f.shape[4] - 20),
)

fig, axes = pplt.subplots(ncols=5, figwidth=7, spanx=False, figheight=None)
for i, ax in enumerate(axes):
    ax.plot(proj[i] / np.max(proj[i]), color='black')
    ax.axvspan(crop[i][0], crop[i][1] - 1, alpha=0.1, color='black')
    ax.format(xlabel=dims[i] + ' [pixel]')
# axes.format(yscale='log')
plt.show()

Make temporary arrays `_f` and `_coords` to see if you like the crop settings before making lots of plots. 

In [ ]:
ind = tuple([slice(*c) for c in crop])
_f = f[ind]
_coords = [c[ind[i]] for i, c in enumerate(coords)]

In [ ]:
_f = _f / np.max(_f)

In [ ]:
_f[_f < 10.0**-3.0 * _f.max()] = 0

## Interactive

### 1D projection of int slice 

In [ ]:
mplt.interactive_proj1d(_f, coords=_coords, dims=dims, units=units,
                        kind='step',
                        slice_type='int', default_ind=4)

### 1D projection of range slice 

In [ ]:
mplt.interactive_proj1d(_f, coords=_coords, dims=dims, units=units,
                        slice_type='range', default_ind=4)

### 2D projection of int slice 

In [ ]:
mplt.interactive_proj2d(_f, coords=_coords, default_ind=(2, 3),
                        slice_type='int', dims=dims, units=units)

### 2D projection of range slice

In [ ]:
mplt.interactive_proj2d(f / f_max, coords=coords, default_ind=(2, 3),
                        slice_type='range', dims=dims, units=units)

Save the grid coordinates if everything looks good.

In [ ]:
f = _f
coords = _coords
utils.save_stacked_array(f'_output/coords_{filename}_cropped.npz', coords)

## Static 

### Projections

In [ ]:
frac_thresh = 10.0**-2.7

Corner plot.

In [ ]:
importlib.reload(mplt)

In [ ]:
for norm in [None, 'log']:
    axes = mplt.corner(
        f,
        coords=coords,
        diag_kind='None',  # {'line', 'None'}
        prof='edges',  # {True, False, 'edges'}
        prof_kws=prof_kws,
        fig_kws=dict(figwidth=(0.85 * 1.5 * 4), space=1),
        labels=["x [mm]", "x' [mrad]", "y [mm]", "y' [mm]", "w [MeV]"],
        norm=norm,
        handle_log='floor',
        thresh=frac_thresh,
        thresh_type='frac',
        linewidth=0, rasterized=True,
        discrete=False,
    )
    label_kw = dict(fontsize='large')
    axes.format(xlabel_kw=label_kw, ylabel_kw=label_kw)
    for png in [False, True]:
        figname = f'_output/int_corner_norm{norm}'
        if png:
            figname += '.png'
        plt.savefig(figname)
    plt.show()

2D projections (this will save a lot of files).

In [ ]:
for i in trange(5):
    for j in trange(i):
        H = utils.project(f, (j, i))
        H = H / np.max(H)
        for norm in [None, 'log']:
            for prof in [True, False]:
                fig, ax = pplt.subplots()
                mplt.plot_image(H, x=coords[j], y=coords[i], ax=ax,
                                profx=prof, profy=prof, prof_kws=prof_kws, 
                                thresh=frac_thresh,
                                thresh_type='frac',
                                norm=norm, handle_log='floor', colorbar=True)
                ax.format(xlabel=dims_units[j], ylabel=dims_units[i])
                plt.savefig(f"_output/proj_{dims[j]}-{dims[i]}_{norm}.png", dpi=300)
                plt.close()

### Slices

#### Max pixel 

In [ ]:
ind = np.unravel_index(np.argmax(f), f.shape)
ind = tuple([i for i in ind])
print('max pixel indices:', ind)

In [ ]:
prof = True
axes_slice = [(k, j, i) for i in range(f.ndim) for j in range(i) for k in range(j)]
axes_view = [tuple([i for i in range(f.ndim) if i not in axis])
             for axis in axes_slice]
for axis, axis_view in zip(axes_slice, axes_view):
    idx = utils.make_slice(5, axis, [ind[i] for i in axis])
    f_slice = f[idx]
    f_slice = f_slice / np.max(f_slice)
    
    dim1, dim2 = [dims[i] for i in axis_view]
    
    fig, plot_axes = pplt.subplots(ncols=2)
    for ax, norm in zip(plot_axes, [None, 'log']):
        mplt.plot_image(f_slice, x=coords[axis_view[0]], y=coords[axis_view[1]],
                        ax=ax,
                        profx=prof, profy=prof, prof_kws=prof_kws, handle_log='floor',
                        thresh=frac_thresh, thresh_type='frac', vmin=0.001, vmax=1.0,
                        norm=norm, colorbar=True)
    plot_axes.format(xlabel=dim1, ylabel=dim2)
    string = '_output/int_slice_'
    for i in axis:
        string += f'{dims[i]}-{ind[i]}'
    _dims = [dims[i] for i in axis]
    _units = [units[i] for i in axis]
    _vals = [coords[i][ind[i]] for i in axis]
    plot_axes.format(suptitle=f'{_dims[0]} = {_vals[0]:.2f} [{_units[0]}],  {_dims[1]} = {_vals[1]:.2f} [{_units[1]}],  {_dims[2]} = {_vals[2]:.2f} [{_units[2]}]')
    plot_axes.format(suptitle_kw=dict(fontweight='normal'))
    plt.savefig(string)
    plt.show()

#### Energy slices

In [ ]:
n = 6
ncols = 6
nrows = int(np.ceil(n / ncols))

wmin = -0.05
wmax = +0.05

lo = np.where(coords[4] < wmin)[0][-1]
hi = np.where(coords[4] > wmax)[0][0]

print(coords[4][lo:hi])
ks = np.linspace(lo, hi, n).astype(int)

In [ ]:
cmap = 'viridis'
for j in range(4):
    for i in range(j):
        if not ((i == 0 and j == 1) or (i == 2 and j == 3)):
            continue
        f3d = utils.project(f, (i, j, 4))
        f3d = f3d / np.max(f3d)
        for norm in [None, 'log']:
                vmin = vmax = None
                # colorbar = 't'
                colorbar = False
                prof = True
                _vmin = vmin if not norm else None
                _vmax = vmax if not norm else None            
                fig, axes = pplt.subplots(ncols=ncols, nrows=nrows, figwidth=9.0)
                for ax, k in zip(axes, ks):      
                    image = f3d[:, :, k]
                    image = image / np.max(image)
                    mplt.plot_image(
                        image, x=coords[i], y=coords[j], ax=ax,
                        norm=norm,
                        linewidth=0, rasterized=True, ec='None',
                        # vmin=_vmin, 
                        vmin=1e-3,
                        vmax=1.0,
                        # colorbar=colorbar, 
                        # colorbar=ax==axes[-1],
                        # colorbar_kw=dict(width=0.085, space=1),
                        profx=prof,
                        profy=prof,
                        prof_kws=prof_kws,
                        handle_log='floor',
                        thresh=frac_thresh,
                        thresh_type='frac',
                        cmap=cmap,
                        discrete=False,
                        contour_kws=dict(color='white', lw=0.3, alpha=0.3, 
                                         values=[0.001, 0.001000001]
                                         # values=[0.001, 0.01, 0.1],
                                        )
                    )
                    ax.annotate(f'w = {coords[4][k]:.3f} [MeV]', xy=(0.02, 0.98), verticalalignment='top',
                                xycoords='axes fraction', fontsize='small', color='white')
                axes.format(xlabel=labels[i], ylabel=labels[j])
                for png in [False, True]:
                    figname = f'_output/energy_slice_proj_{dims[i]}-{dims[j]}_{cmap}_{norm}'
                    if png:
                        figname += '.png'
                    plt.savefig(figname)
                plt.show()

## Covariance matrix

Compute the 5$\times$5 covariance matrix (this will take a while if `f` is large).

In [ ]:
Sigma, means = ba.dist_cov(f, coords, disp=True)
sympy.Matrix(np.round(Sigma, 3))

Compute the $5 \times 5$ correlation matrix from the covariance matrix.

In [ ]:
Corr = utils.cov2corr(Sigma)
sympy.Matrix(np.round(Corr, 3))

In [ ]:
g = sns.heatmap(Corr, xticklabels=dims, yticklabels=dims, annot=True,
                cbar=False, cmap='grays')
plt.savefig('_output/correlation_matrix.png')

In [ ]:
np.savetxt('_output/Sigma.dat', Sigma)
np.savetxt('_output/Corr.dat', Corr)

In [ ]:
for i in range(5):
    for j in range(i):
        angle, cx, cy = ba.rms_ellipse_dims(Sigma[j, j], Sigma[i, i], Sigma[j, i])
        center = (means[j], means[i])
        width = 4.0 * cx
        height = 4.0 * cy

        fig, ax = pplt.subplots()
        mplt.plot_image(utils.project(f, (j, i)), x=coords[j], y=coords[i], ax=ax,
                        frac_thresh=frac_thresh)
        ax.add_patch(
            patches.Ellipse(
                center, width, height, angle=-np.degrees(angle),
                ec='white', fill=False, lw=0.9, alpha=0.6,
            )
        )
        ax.format(xlabel=dims_units[i], ylabel=dims_units[j])
        plt.savefig(f'_output/rms_ellipse_{dims[j]}-{dims[i]}.png')

In [ ]:
axes = mplt.corner(
    f,
    coords=coords,
    prof=False,
    labels=dims_units,
    frac_thresh=frac_thresh,
)
for i in range(5):
    for j in range(i):
        ax = axes[i, j]
        angle, cx, cy = ba.rms_ellipse_dims(Sigma[j, j], Sigma[i, i], Sigma[j, i])
        center = (means[j], means[i])
        width = 4.0 * cx
        height = 4.0 * cy
        angle = -np.degrees(angle)
        ax.add_patch(patches.Ellipse(center, width, height, angle=angle,
                                     ec='white', fill=False, lw=0.7, alpha=0.6))
plt.savefig('_output/int_corner_cov.png')
plt.show()

In [ ]:
alpha_x, alpha_y, beta_x, beta_y = ba.twiss(Sigma)
eps_x, eps_y, eps_1, eps_2 = ba.emittances(Sigma)
print(f'alpha_x = {alpha_x}')
print(f'alpha_y = {alpha_y}')
print(f'beta_x = {beta_x}')
print(f'beta_y = {beta_y}')
print(f'epsx = {eps_x}')
print(f'epsy = {eps_y}')
print(f'eps1 = {eps_1}')
print(f'eps2 = {eps_2}')

### Root-mean-square emittances vs. energy

Computing 4D covariance matrix for each energy slice also takes a long time. For now, compute x-x' and y-y' emittances.

In [ ]:
emittances, twiss = [], []
for (i, j) in [(0, 1), (2, 3)]:
    f3d = utils.project(f, (i, j, 4))
    _Sigmas = np.zeros((shape[4], 2, 2))  # x-xp covariance matrix
    _means = np.zeros((shape[4], 2))  # x-xp mean
    _emittances = np.zeros(shape[4])  # rms emittance
    _twiss = np.zeros((shape[4], 2))  # rms alpha, beta
    for k in trange(shape[4]):  # for each energy w
        _Sigmas[k], _means[k] = ba.dist_cov(f3d[:, :, k], [coords[i], coords[j]])
        _emittances[k] = ba._emittance(_Sigmas[k])
        _twiss[k] = ba._twiss(_Sigmas[k])
    emittances.append(_emittances)
    twiss.append(_twiss)
emittances = np.array(emittances).T
twiss = np.hstack(twiss)

In [ ]:
colors = pplt.Cycle('colorblind').by_key()['color']
labels = [r'$\varepsilon_x$', r'$\varepsilon_y$']

fig, ax = pplt.subplots(figsize=(4.5, 2.5))
for i in range(2):
    ax.plot(coords[4], emittances[:, i], label=labels[i], marker='.', ms=3)
    ax.axhline([eps_x, eps_y][i], color=colors[i], label=labels[i]+' (full)', 
               alpha=0.3, ls='-')
ax.format(ylabel='[mm mrad]', xlabel='w [MeV]', title='Energy slice emittances')
ax.legend(ncols=1, loc='r')
plt.savefig('_output/energy_slice_emittances.png')

In [ ]:
twiss_labels = [r'$\alpha_x$', r'$\beta_x$', r'$\alpha_y$', r'$\beta_y$']
cut = 20
idx = np.arange(cut, shape[4] - cut)

fig, ax = pplt.subplots(figsize=(4.5, 2.5))
for i, _alpha in zip((0, 2), [alpha_x, alpha_y]):
    ax.plot(coords[4][idx], twiss[idx, i], label=twiss_labels[i], marker='.', ms=3)
    ax.axhline(_alpha, color=colors[i-1], label=twiss_labels[i]+' (full)', 
               alpha=0.3, ls='-')
ax.format(xlabel='w [MeV]', title='Energy slice rms alpha')
ax.legend(ncols=1, loc='r')
plt.savefig('_output/energy_slice_alphas.png')

In [ ]:
fig, ax = pplt.subplots(figsize=(4.5, 2.5))
for i, _beta, color in zip((1, 3), [beta_x, beta_y], colors[:2]):
    ax.plot(coords[4][idx], twiss[idx, i], label=twiss_labels[i], marker='.', ms=3)
    ax.axhline(_beta, color=colors[i-1], label=twiss_labels[i]+' (full)', 
               alpha=0.3, ls='-')
ax.format(xlabel='w [MeV]', title='Energy slice rms beta')
ax.legend(ncols=1, loc='r')
plt.savefig('_output/energy_slice_betas.png')